<a href="https://colab.research.google.com/github/Hamza-Atiq/PIAIC-Q2/blob/main/piaic_quarter2_lecture_5_tool_calling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install -qU langchain langchain-community langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 66.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.5/41.5 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.2 MB/s eta 0:00:00


In [ ]:
import os
from google.colab import userdata
os.environ['GOOGLE_API_KEY'] = userdata.get('gemini')

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.tools import tool
from langchain.tools import Tool
from langchain.agents import initialize_agent, AgentType
from langchain.agents import AgentExecutor

llm = ChatGoogleGenerativeAI(model = 'gemini-2.0-flash-exp' , verbose= True)

In [ ]:
@tool
def multiply(a : int , b : int) ->int:
    """Multiply a and b.

    Args:
        a: first int
        b: second int
    """
    return a*b

llm_with_tools = llm.bind_tools([multiply])

In [ ]:
result = llm_with_tools.invoke("my name is Hamza Atiq")

In [ ]:
result

AIMessage(content="Hello Hamza Atiq, it's nice to meet you!", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-81638afb-719c-44b2-ab93-bd477c5dbbee-0', usage_metadata={'input_tokens': 69, 'output_tokens': 14, 'total_tokens': 83, 'input_token_details': {'cache_read': 0}})

In [ ]:
result1 = llm_with_tools.invoke("what is 2*3")

In [ ]:
result1

AIMessage(content='', additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"a": 2.0, "b": 3.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-dabfcf43-e219-4847-b50a-eeef46e8fec5-0', tool_calls=[{'name': 'multiply', 'args': {'a': 2.0, 'b': 3.0}, 'id': '3b5bd47a-b666-4767-a2a1-3e12c91f6572', 'type': 'tool_call'}], usage_metadata={'input_tokens': 69, 'output_tokens': 3, 'total_tokens': 72, 'input_token_details': {'cache_read': 0}})

In [ ]:
print(result1)

content='' additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"a": 2.0, "b": 3.0}'}} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}]} id='run-dabfcf43-e219-4847-b50a-eeef46e8fec5-0' tool_calls=[{'name': 'multiply', 'args': {'a': 2.0, 'b': 3.0}, 'id': '3b5bd47a-b666-4767-a2a1-3e12c91f6572', 'type': 'tool_call'}] usage_metadata={'input_tokens': 69, 'output_tokens': 3, 'total_tokens': 72, 'input_token_details': {'cache_read': 0}}


In [ ]:
def add_numbers(num1, num2):
    return num1 + num2

def subtract_numbers(num1, num2):
    return num1 - num2

def multiply_numbers(num1, num2):
    return num1 * num2

def divide_numbers(num1, num2):
    if num2 == 0:
        return "Cannot divide by zero"
    return num1 / num2

def square_number(num):
    return num ** 2

def cube_number(num):
    return num ** 3

def square_root(num):
    if num < 0:
        return "Cannot calculate square root of a negative number"
    return num ** 0.5

def factorial(num):
    if num < 0:
        return "Factorial is not defined for negative numbers"
    elif num == 0:
        return 1
    else:
        result = 1
        for i in range(1, num + 1):
            result *= i
        return result


In [ ]:
addition_tool = Tool(
    name = "add_numbers",
    func = lambda x: f"The result is: {add_numbers(*map(float , x.split()))}",
    description = "Adds two numbers. Input should be two integers seperated with space"
)

subtraction_tool = Tool(
    name = "subtract_numbers",
    func = lambda x: f"The result is: {subtract_numbers(*map(float , x.split()))}",
    description = "Subtracts two numbers. Input should be two float values if applicable and seperated with space. result will be two decimal point"
)

multiplication_tool = Tool(
    name = "multiply_numbers",
    func = lambda x: f"The result is: {multiply_numbers(*map(float , x.split()))}",
    description = "Multiplies two numbers. Input should be two integers seperated with space"
)

division_tool = Tool(
    name = "divide_numbers",
    func = lambda x: f"The result is: {divide_numbers(*map(float , x.split()))}",
    description = "Divides two numbers. Input should be two integers seperated with space"
)

square_tool = Tool(
    name = "square_number",
    func = lambda x: f"The result is: {square_number(float(x))}",
    description = "Squares a number. Input should be an integer"
)

cube_tool = Tool(
    name = "cube_number",
    func = lambda x: f"The result is: {cube_number(float(x))}",
    description = "Cubes a number. Input should be an integer"
)

square_root_tool = Tool(
    name = "square_root",
    func = lambda x: f"The result is: {square_root(float(x))}",
    description = "Calculates the square root of a number. Input should be an integer"
)

factorial_tool = Tool(
    name = "factorial",
    func = lambda x: f"The result is: {factorial(int(x))}",
    description = "Calculates the factorial of a number. Input should be an integer"
)



In [ ]:
tools = [addition_tool , subtraction_tool,
         multiplication_tool , division_tool,
         square_tool , cube_tool,
         square_root_tool , factorial_tool]

agent = initialize_agent(
    tools = tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

<ipython-input-12-c615661933c5>:6: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


In [ ]:
response1 = agent.run("Add 2 and 4" )
print(response1)





> Entering new AgentExecutor chain...


<ipython-input-13-d82915abd626>:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response1 = agent.run("Add 2 and 4" )


I need to add two numbers.
Action: add_numbers
Action Input: 2 4
Observation: The result is: 6.0
Thought:I now know the final answer
Final Answer: 6.0

> Finished chain.
6.0


In [ ]:
response2 = agent.run("what is 10-6.9")
print(response2)



> Entering new AgentExecutor chain...
I need to subtract two numbers.
Action: subtract_numbers
Action Input: 10 6.9
Observation: The result is: 3.0999999999999996
Thought:I now know the final answer
Final Answer: 3.10

> Finished chain.
3.10


In [ ]:
response3 = agent.run("what is 66*777")
print(response3)



> Entering new AgentExecutor chain...
I need to multiply 66 and 777.
Action: multiply_numbers
Action Input: 66 777
Observation: The result is: 51282.0
Thought:I now know the final answer
Final Answer: 51282

> Finished chain.
51282


In [ ]:
response4 = agent.run("what is 567/9")
print(response4)



> Entering new AgentExecutor chain...
I need to divide 567 by 9.
Action: divide_numbers
Action Input: 567 9
Observation: The result is: 63.0
Thought:I now know the final answer
Final Answer: 63.0

> Finished chain.
63.0


In [ ]:
response5 = agent.run("what is square of 4 is?")
print(response5)



> Entering new AgentExecutor chain...
I need to calculate the square of 4.
Action: square_number
Action Input: 4
Observation: The result is: 16.0
Thought:I now know the final answer
Final Answer: 16

> Finished chain.
16


In [ ]:
response6 = agent.run("What is cube of 3 is?")
print(response6)



> Entering new AgentExecutor chain...
I need to calculate the cube of 3.
Action: cube_number
Action Input: 3
Observation: The result is: 27.0
Thought:I now know the final answer
Final Answer: 27.0

> Finished chain.
27.0


In [ ]:
response7 = agent.run("what is square root of 25?")
print(response7)



> Entering new AgentExecutor chain...
I need to calculate the square root of 25.
Action: square_root
Action Input: 25
Observation: The result is: 5.0
Thought:I now know the final answer
Final Answer: 5.0

> Finished chain.
5.0


In [ ]:
response8 = agent.run("what is factorial of 5?")
print(response8)



> Entering new AgentExecutor chain...


I need to calculate the factorial of 5.
Action: factorial
Action Input: 5
Observation: The result is: 120
Thought:I now know the final answer
Final Answer: 120

> Finished chain.
120


In [ ]:
class Calculator:
    def calculate(self, expression: str) -> str:
        try:
            # Use Python's eval to compute the result safely
            result = eval(expression, {"__builtins__": None}, {})
            return str(result)
        except Exception as e:
            return f"Error: {e}"

In [ ]:
from langchain_core.tools import tool

@tool
def calculator(expression: str) -> str:
    """
    Perform arithmetic calculations.
    Input: A mathematical expression as a string (e.g., "2 + 2").
    Output: Result of the calculation as a string.
    """
    calc = Calculator()
    return calc.calculate(expression)

In [ ]:
from langchain.tools import Tool



# Task weather API key

In [ ]:
! pip install -qU requests

In [ ]:
import requests
OPENWEATHER_API_KEY = userdata.get('weatherapikey')

In [ ]:
def fetch_weather(city_name):

      if not OPENWEATHER_API_KEY:
        return "Error: Missing OpenWeatherMap API key."

      url = 'https://api.openweathermap.org/data/2.5/weather'

      params = {
          'q' : city_name,
          'appid' : OPENWEATHER_API_KEY,
          'units' : 'metric'
      }

      try:
        response = requests.get(url , params = params)
        response.raise_for_status()
        weather_data = response.json()

        temperature = weather_data['main']['temp']
        description = weather_data['weather'][0]['description']
        return f"The temperature in {city_name} is {temperature}°C and the weather is {description}."

      except Exception as e:
        return f"An error occurred while fetching weather data: {e}"


In [ ]:
weather_tool = Tool(
    name = "fetch_weather",
    func = fetch_weather,
    description = "Fetch the weather information for a given city using OpenWeatherMap. Input should be a city name."
)

In [ ]:
def main():

    tools = [weather_tool ]
    agent = initialize_agent(
        tools = tools,
        llm=llm,
        agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
        verbose=True
    )

    while True:

        user_input = input("Please Enter City Name or exit : ")

        if user_input.lower().strip() == 'exit':
            print("Exiting the program.")
            break

        response = agent.run(user_input)
        print(response)



In [ ]:
if __name__ == "__main__":
    main()

Please Enter City Name or exit : rawalpindi


> Entering new AgentExecutor chain...
I need to get the weather information for Rawalpindi.
Action: fetch_weather
Action Input: Rawalpindi
Observation: The temperature in Rawalpindi is 16.8°C and the weather is overcast clouds.
Thought:I now know the final answer
Final Answer: The temperature in Rawalpindi is 16.8°C and the weather is overcast clouds.

> Finished chain.
The temperature in Rawalpindi is 16.8°C and the weather is overcast clouds.
Please Enter City Name or exit : exit
Exiting the program.


In [ ]:
OPENWEATHER_API_KEY = userdata.get('weatherapikey')

def fetch_weather_data(city_name: str) -> dict:
    """
    Fetch current weather data for the specified city using OpenWeatherMap API.
    Returns a dictionary with weather info, or raises an exception on error.
    """
    if not OPENWEATHER_API_KEY:
        raise ValueError("Missing OpenWeatherMap API key. Please set OPENWEATHER_API_KEY.")

    url = "https://api.openweathermap.org/data/2.5/weather"
    params = {
        "q": city_name,
        "appid": OPENWEATHER_API_KEY,
        "units": "metric"  # 'metric' for Celsius, 'imperial' for Fahrenheit
    }

    response = requests.get(url, params=params)
    response.raise_for_status()  # Raises HTTPError for 4xx/5xx
    return response.json()

def generate_linkedin_post(city_name: str) -> str:
    """
    Fetches weather data for a given city and then formats it into a
    LinkedIn-friendly post.
    """
    try:
        data = fetch_weather_data(city_name)

        # Extract relevant info from the response
        temperature = data["main"]["temp"]
        humidity = data["main"]["humidity"]
        weather_description = data["weather"][0]["description"].capitalize()
        feels_like = data["main"]["feels_like"]

        # You could also get wind speed, cloudiness, etc. if you like:
        # wind_speed = data["wind"]["speed"]
        # cloudiness = data["clouds"]["all"]

        # Build a LinkedIn post with emojis, hashtags, and your own style
        post_text = (
            f"☀️ Weather Update: {city_name} ☀️\n\n"
            f"It's currently **{temperature}°C** and **{weather_description}** "
            f"(feels like {feels_like}°C). Humidity is at **{humidity}%**.\n\n"
            f"Whether you're in {city_name} or elsewhere, stay prepared! "
            f"Don't forget to stay hydrated if it's hot, or bundle up if it's chilly.\n\n"
            f"#Weather #LinkedInUpdate #StaySafe #OpenWeather"
        )

        return post_text

    except requests.exceptions.HTTPError as http_err:
        return (f"Oops! Could not fetch weather data for '{city_name}'. "
                f"HTTP error occurred: {str(http_err)}.")
    except Exception as e:
        return (f"An error occurred while fetching weather data for '{city_name}': {str(e)}")

def main():
    """
    Simple CLI loop to show how you'd generate a LinkedIn post
    for any city the user enters.
    """
    while True:
        city_name = input("Enter a city name (or 'exit' to quit): ")
        if city_name.lower().strip() == "exit":
            print("Exiting the program.")
            break

        post_text = generate_linkedin_post(city_name)
        print("\n--- LinkedIn Post Preview ---")
        print(post_text)
        print("----------------------------\n")

if __name__ == "__main__":
    main()


Enter a city name (or 'exit' to quit): rawalpindi

--- LinkedIn Post Preview ---
☀️ Weather Update: rawalpindi ☀️

It's currently **16.8°C** and **Overcast clouds** (feels like 16.1°C). Humidity is at **60%**.

Whether you're in rawalpindi or elsewhere, stay prepared! Don't forget to stay hydrated if it's hot, or bundle up if it's chilly.

#Weather #LinkedInUpdate #StaySafe #OpenWeather
----------------------------

Enter a city name (or 'exit' to quit): exit
Exiting the program.
